In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!git clone https://github.com/thepineapplepirate/quantumcomputing_GCC2024.git
!cp -r quantumcomputing_GCC2024/qufold/ .

In [ ]:
import sys

sys.path.append("")

from qiskit.circuit.library import RealAmplitudes, EfficientSU2
from qiskit_algorithms import VQE, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA, SLSQP, L_BFGS_B, SPSA
# from qiskit.primitives import Sampler as LocalSampler
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler, Estimator
import matplotlib.pyplot as plt
import numpy as np
from qufold import (
    MiyazawaJerniganInteraction,
    Peptide,
    ProteinFoldingProblem,
    PenaltyParameters,
)
from qiskit.quantum_info import SparsePauliOp

In [ ]:
def build_pf(main_seq: str):
    """Builds the protein folding problem for the given sequence."""
    # Define the interaction
    side_chains = [""] * len(main_seq)

    mj_interaction = MiyazawaJerniganInteraction()

    penalty_back = 10
    penalty_chiral = 10
    penalty_1 = 10

    penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)

    peptide = Peptide(main_seq, side_chains)

    protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)

    return protein_folding_problem

In [ ]:
# Here we define the primary amino acid sequence
# the following lines automate the retrieval of the primary amino acid sequence
# from the Galaxy inputs, either pasted in or from a FASTA file in your history
main_chains = []
for galaxy_input in GALAXY_INPUTS:
    if isinstance(GALAXY_INPUTS[galaxy_input], str):
        main_chains.append(GALAXY_INPUTS[galaxy_input])    
    else:
        for dataset in GALAXY_INPUTS[galaxy_input]:
            protein_sequence = dataset['path']
            with open(protein_sequence) as inputfile:
                main_chains.append(inputfile.readlines()[1].strip())

main_chain = main_chains[0]
print(main_chain)
side_chains = [""] * len(main_chain)

In [ ]:
pf = build_pf(main_chain) #creates the PF problem instance
qubit_op = pf.qubit_op() #creates the problem Hamiltonian

In [ ]:
aer_backend_options = {"device": "CPU"}
estimator = Estimator(backend_options=aer_backend_options)
sampler = Sampler(backend_options=aer_backend_options)

def run_vqe(qubit_op):
    # set classical optimizer
    optimizer = COBYLA(maxiter=150)

    # set variational ansatz
    ansatz = RealAmplitudes(reps=1)

    counts = []
    values = []

    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    vqe = SamplingVQE(
        sampler,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=store_intermediate_result,
    )

    raw_result = vqe.compute_minimum_eigenvalue(qubit_op)

    return raw_result, counts, values

In [ ]:
raw_result, counts, values = run_vqe(qubit_op)

In [ ]:
fig = plt.figure()

plt.plot(counts, values)
plt.ylabel("Conformational Energy")
plt.xlabel("VQE Iterations")
plt.show()

In [ ]:
result = pf.interpret_new(raw_result)

In [ ]:
print(
    "The bitstring representing the shape of the protein during optimization is: ",
    result.turn_sequence,
)
print("The expanded expression is:", result.get_result_binary_vector())

##

print(f"The folded protein's main sequence of turns is: {result.protein_shape_decoder.main_turns}")
print(f"and the side turn sequences are: {result.protein_shape_decoder.side_turns}")

fig = result.get_figure(title="3dcrd", ticks=False, grid=True)
fig.get_axes()[0].view_init(10, 70)

In [ ]:
result.save_xyz_file(replace=True)
!mv *.xyz outputs/collection/
put(main_chain + ".xyz")